In [ ]:
# Initial imports
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
from datetime import datetime
import numpy as np
#import tensorflow as tf

#  Import and read the file.
supply_df = pd.read_csv("C:/Users/Elissa/UCB/dataVizfinal/SupplyData/CAISO-ALL-SUPPLY-DATA2.csv")
supply_df.head()


In [ ]:
supply_df['Sum'] = supply_df['Renewables'] + supply_df['Large hydro'] + supply_df['Imports'] + supply_df['Batteries'] + supply_df['Nuclear'] + supply_df['Coal'] + supply_df['Other'] + supply_df['Natural gas']


In [ ]:
supply_df.dtypes

In [ ]:
baby_df = supply_df['Time'].str.split(":", expand=True)
print(baby_df)

In [ ]:
baby_df[0] = baby_df[0].astype(int)
baby_df[1] = baby_df[1].astype(int)

In [ ]:
baby_df= baby_df.drop([2],1)

In [ ]:
baby_df.dtypes

In [ ]:
# Rename columns and create years value for calculating demand_EV
baby_df = baby_df.rename(columns={baby_df.columns[0]: 'Hours'})
baby_df = baby_df.rename(columns={baby_df.columns[1]: 'Minutes'})

In [ ]:
supply_df['Date'] = pd.to_datetime(supply_df['Date'])

#Create a new column for'Year'
baby_df['Year'] = supply_df['Date'].dt.year
#Convert to integers to be used in computations
baby_df['Year'] = baby_df['Year'].astype(int)

baby_df.head(100)

In [ ]:
#Make a date string column 
date_df = pd.DataFrame()
supply_df['Date1']=supply_df['Date'].astype(str)

date_df = supply_df['Date1'].str.split("-", expand=True)
date_df['DateAttribute'] = date_df[1] + date_df[2]
date_df = date_df.drop([0,1,2],1)


In [ ]:
baby_df['DemandMW'] = 0.0

mask = (baby_df['Year'] == 2018) & (baby_df['Hours']<=6) | (baby_df['Hours']>=22)
mask1 = (baby_df['Year'] == 2019) & (baby_df['Hours']<=6) | (baby_df['Hours']>=22)
mask2 = (baby_df['Year'] == 2020) & (baby_df['Hours']<=6) | (baby_df['Hours']>=22)


baby_df.loc[mask,'DemandMW'] = 6.2356
baby_df.loc[mask1,'DemandMW'] = 8.2818
baby_df.loc[mask2,'DemandMW'] = 9.9174

In [ ]:
calc_df = pd.merge(baby_df, date_df, left_index=True, right_index=True, how='outer')
calc_df.head(-5)

In [ ]:
calc_df.dtypes


In [ ]:
def fun(datetimething):
     return datetimething.timetuple().tm_yday

In [ ]:
supply_df['Date'] = supply_df['Date'].apply(fun)
supply_df.head(1300)

In [ ]:
supply_df = supply_df.drop(['Date1'],1)
final_df = pd.merge(supply_df, calc_df, left_index=True, right_index=True, how='outer')


In [ ]:
df1= pd.DataFrame()

In [ ]:
df1 = final_df.groupby(['DateAttribute','Time'])['Sum'].agg('mean')
df1 = df1.reset_index()
df1 = df1.rename(columns={df1.columns[2]: 'AvgSUM(3yr)'}).round(2)

print(len(df1))
df1.dtypes

In [ ]:
df1['Date1']= "2020" + df1['DateAttribute']
df1['pDateTime'] = df1['Date1'] +" " +df1['Time']
df1['Date2']= pd.to_datetime(df1['pDateTime'])
df1=df1.set_index('Date2')       
                                            
df1=df1.sort_index(ascending=True)
df1=df1.reset_index()
df1=df1.drop(['Date2','Date1','pDateTime'],1)
df1

In [ ]:
z = df1.loc[(df1['DateAttribute'] == '0229')]
print(z.index)


In [ ]:
df1 = df1.drop(df1.index[16992:17280])
df1 = df1.reset_index(drop=True)
print(len(df1))
df1

In [ ]:
x = df1.loc[(df1['DateAttribute'] == '0410')]
print(x.index)

print(df1.loc[28511:28900])



In [ ]:
final_df = final_df.set_index('DateTime')       
                                            
final_df=final_df.sort_index(ascending=True)
final_df=final_df.reset_index()
final_df


In [ ]:
y = final_df.loc[(final_df['DateAttribute'] == '0229')]
print(y.index)

In [ ]:
#drop leap year values
final_df = final_df.drop(final_df.index[197257:197545])
print(len(final_df))

In [ ]:
#print(type(final_df))
print(final_df.loc[197256:197560])

In [ ]:
copy1_df1 = df1.copy()
copy2_df1 = df1.copy()


In [ ]:
new_df1 = df1.append(copy1_df1).reset_index(drop=True)

In [ ]:
new_df2 = new_df1.append(copy2_df1)
new_df2 = new_df2.reset_index(drop=True)
print(len(new_df2))
new_df2

In [ ]:
new_df2 = new_df2.drop(new_df2.index[0:28512]).reset_index(drop=True)
new_df2
